##### 一、向量的内积 vs 矩阵的乘积 

- 向量的内积(两个向量的相似度度量:np.dot): $\boldsymbol{x} \cdot \boldsymbol{y} = x_1y_1 + x_2y_2 + \cdots +x_ny_n$
- 矩阵的乘积（权重矩阵对数据矩阵的线性变换:np.dot）：
$$
\boldsymbol{C} = \underbrace{\boldsymbol{A}}_{数据样本矩阵}  \times \underbrace{\boldsymbol{B}}_{权重矩阵}

\longrightarrow  

\boldsymbol{A}矩阵的行（样本）通过\boldsymbol{B}矩阵的列（变换）进行处理。即\boldsymbol{B}矩阵定义了如何对\boldsymbol{A}矩阵的每个样本进行线性变换

{\color{Red} \boldsymbol{A}与\boldsymbol{B}交换顺序也行哈，本质还是运算的线性作用要分清！} 
$$
- 矩阵的哈达玛积(逐元素积：A * B): 1对1帮扶

####  二、神经网络的推理过程

![](./attachements/神经网络的推理过程-神经元表示法.png)
![](./attachements/神经网络的推理过程-层表示法.png)

注意：

- 所有的层都有 forward() 方法和 backward() 方法
- 所有的层都有 params 和 grads 实例变量


In [1]:
import numpy as np

In [9]:
# 层-Affine

class Affine:

    def __init__(self, W, b) -> None:
        self.params = [W, b]

    def forward(self, x):
        W,b = self.params
        out = np.dot(x, W) + b
        return out

    def backward(self):
        pass

In [3]:
# 层-Sigmoid

class Sigmoid:
    
    def __init__(self) -> None:
        self.params = []

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        return out
    
    def backward(self):
        ...

In [11]:
# Neural Network - TwoLayerNet

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size) -> None:
        
        I, H, O = input_size, hidden_size, output_size

        # 初始化权重和偏置
        W1 = np.random.randn(I, H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H, O)
        b2 = np.random.randn(O)


        # 生成层
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),    # 秒1
            Affine(W2, b2)
        ]

        # 将所有权重整理到列表中
    

    def predict(self, x):
        # 实现前向传播
        for layer in self.layers:
            x = layer.forward(x) # 秒2
        return x
    


x = np.random.randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = model.predict(x)
print(x)
print(" * " * 25)
print(s)

[[ 1.56104902  0.32854007]
 [ 1.27436108 -0.93733702]
 [ 1.27306454  0.01473727]
 [ 1.70195476 -0.55325669]
 [-0.10876628  1.32105778]
 [ 0.0033703  -0.09225441]
 [-0.18275277  1.03049557]
 [ 0.71801996  0.44650455]
 [-1.17649957  0.06730696]
 [ 0.66463137  1.73480992]]
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
[[-0.20434097 -0.26609517  2.15629652]
 [-0.21039606 -0.25700322  1.80718516]
 [-0.09941655 -0.19731537  1.99742643]
 [-0.37184777 -0.35307236  2.02179563]
 [ 1.36129906  0.57095152  1.34675316]
 [ 1.12983928  0.46295758  1.15509034]
 [ 1.38957938  0.5914602   1.26418012]
 [ 0.42764283  0.08412406  1.7646761 ]
 [ 1.63048714  0.73735506  0.8925275 ]
 [ 0.59675533  0.1467331   1.95970569]]


#### 三、神经网络的学习

- 损失函数: 神经网络当前学习到的参数好坏怎么度量？
- 导数和梯度: 导数就是当自变量逼近时因变量的变化趋势，梯度是针对复合函数的，每一个自变量都有自己的导数！此为偏导数。
- 链式法则：复合函数中引入的概念爷爷对孙子的导数 = 爷爷对爸爸的导数 x 爸爸对儿子的导数
- 计算图：将输入数据 通过节点（操作）和 边（数据流）构建的图像结构，来表示运算的逻辑链路图
- 反向传播：把神经网络看做是一个多层的复合函数，正向传播是逐层传递激活值，而反向传播是逐层传递损失函数对该层中节点的梯度，用来指导下一次正向传播

##### 3.1 矩阵的乘积实现

![](./attachements/矩阵的乘积-正反向传播计算图.png)
![](./attachements/矩阵的乘积-根据矩阵的形状推导反向传播式.png)

In [12]:
# MatMul 矩阵乘法层实现

class MatMul:

    def __init__(self, W) -> None:
        self.params = W
        self.grads = [np.zeros_like(W)]
        self.x = None

    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x
        return out

    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)
        dw = np.dot(self.x.T, dout)

        self.grads[0][...] = dw # 深拷贝
        return dx
    
    

##### 3.2 带反向传播的Sigmoid层补全

![](./attachements/Sigmoid层的反向传播计算图.png)

In [ ]:
# 带反向传播的Sigmoid层

class Sigmoid:

    def __init__(self) -> None:
        self.params, self.grads = [], []
        self.out = None

    def forward(self, x):
        return 1 / (1 + np.exp(-x))

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

##### 3.3 带反向传播的Affine层的实现

![](./attachements/带反向传播的Affine层的实现.png)

In [13]:
# 带反向传播的Affine层实现

class Affine:

    def __init__(self, W, b) -> None:
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out

    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

##### 3.4 Softmax with loss层的实现

![](./attachements/Softmax-with-Loss层的计算图.png)

In [15]:
# Softmax with loss层的实现

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T

    x = x - np.max(x)  # 溢出对策
    return np.exp(x) / np.sum(np.exp(x))



def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 监督数据是one-hot-vector的情况下，转换为正确解标签的索引
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None  # softmax的输出
        self.t = None  # 监督数据

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:  # 监督数据是one-hot-vector的情况
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size

        return dx

##### 3.5 SGD 俗称 Optimizer实现



In [16]:
# SGD

class SGD:
    def __init__(self, lr=0.01) -> None:
        self.lr = lr

    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

    

##### 3.6 测试伪代码

model = TwoLayerNet(2, 4, 3)
optimizer = SGD()

for i in  range(1000):
    x_batch, t_batch = get_mini_batch(...) # 获取 mini-batch
    loss = model.forward(x_batch, t_batch)
    model.backward()
    optimizer.update(model.params, model.grads)

#### 四、计算的高速化

- 位精度：NumPy 的浮点数默认使用 64 位的数据类型。使用 32 位浮点数也可以无损地（识别精度几乎不下降）进行神经网络的推理和学习。从内存的角度来看，因为 32 位只有 64 位的一半，所以通常首选 32 位。

- cupy:CuPy 和 NumPy 几乎拥有共同的 API，但是使用了GPU更快。不过我的mac是不行的，需要英伟达的GPU


In [ ]:
import cupy as cp 

x = cp.arange(6).reshape(2, 3).astype('f')